Агент - LLM с инструменты. Инструменты - функции из питона.
Например LLM сможет выйти в интернет или получить доступ к калькулятору.
Не все модели могут быть агентами

Реализован агент с UI, созданием заметок (тхт файлы которые агент будет иногда просматривать), поиск информации в сети

Использованные библиотеки
gigachain - создание агента
duckduckgo-search - поиск в сети
gradio - UI
youtube-search - поиск видео по запросу
youtube-search-python - извлечение метаданныех из ролика
youtube-transcript-api - извлечение субтитров

In [1]:
from google.colab import userdata

In [2]:
auth = userdata.get('SBER_AUTH')

In [23]:
!pip install -q gigachain

!pip install -q duckduckgo-search
!pip install -q youtube-transcript-api
!pip install -q youtube-search
!pip install -q youtube-search-python
!pip install -q gradio
!pip install -q gradio
!pip install -q langchain-community


In [24]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

In [25]:
giga = GigaChat(credentials=auth,
                model='GigaChat',
                verify_ssl_certs=False)

Простой пример как это работает
Инструменты - функции питона. Необходимо указать дукоратор @tool и дать детальное описание функций инструмента и как он выполняет функции. На это описание модель будет опираться при выборе и вызове инструмента

In [26]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
  '''Складывает числа a и b'''
  return a + b


@tool
def multiply(a: int, b: int) -> int:
  '''Умножает a на b'''
  return a * b


tools = [add, multiply]

Связываем ллм с инструментами

In [37]:
from langchain.agents import AgentExecutor, gigachat_functions_agent


from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

agent = gigachat_functions_agent(giga, tools)

#AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

TypeError: 'module' object is not callable

При передаче запроса модели, она может выбрать инструмент по своему усмотрению

In [ ]:
query = 'Сколько будет 3*12?'
llm_with_tools.invoke(query) # тут уже цепь и обращение к этой цепи через invoke

Бахнем интрументы работающие с заметками и поиском информации в интернете и ютабе.
Далее инструмент для заметок

In [ ]:
!mkdir Заметки
from  langchain_community.agent_toolkits import FileManagementToolkit # предоставляет доступ к папке
#имеется 6 инструментов
file_tools = FileManagementToolkit(root_dir='/content/Заметки')


In [ ]:
file_tools.get_tools()

[CopyFileTool(root_dir='/content/Заметки'),
 DeleteFileTool(root_dir='/content/Заметки'),
 FileSearchTool(root_dir='/content/Заметки'),
 MoveFileTool(root_dir='/content/Заметки'),
 ReadFileTool(root_dir='/content/Заметки'),
 WriteFileTool(root_dir='/content/Заметки'),
 ListDirectoryTool(root_dir='/content/Заметки')]

In [ ]:
file_tools.get_tools()[-1].invoke({}).split('\n')
# в директории будет служебный скрытый файл
# хз кто он такой

['No files found in directory .']

In [ ]:
from langchain_core.tools import tool

@tool
def get_list_of_notes() -> str:
  '''
  Возвращает список файлов в папке заметок. В названии заметок \
  содержится тема заметки и её порядковый номер.
  '''

  check_notes = file_tools.get_tools()[-1]
  return check_notes.invoke({})
# invoke - обращение к инструменты
# {} - передаем инструменту словарь с аргументами
# словарь передаётся всем инструментам из этого списка
# но не всем нужны аргументы

In [ ]:
# создаём заметку

@tool
def write_note(topic: str, text: str) -> str:
  '''
  Записывает текст в заметку. Нужно передать тему в качестве части названия \
  будущего файла заметки (дата и время прибавляются к названию автоматически) \
  и текст самой заметки. В качестве ответа функция возвращает сообщение о
  успешной записи.
  '''
  writer_tool = file_tools.get_tools()[-2]
  check_notes = file_tools.get_tools()[-1]
  n = len(check_notes.invoke({}).split('\n'))
  return writer_tool.invoke({'file_path': topic+f'{ n}'+'.txt', 'text': text})

In [ ]:
# читаем файл

@tool
def read_note(file_name: str) -> str:
  '''
  Возвращает содержимое заметки в ответ на заданное имя файла. Если имя указано\
  неправильно, вернет сообщение об ошибке. Имя файла заметки имеет вид\
  "Тема заметки N.txt".
  '''
  read_tool = file_tools.get_tools()[-3]
  try:
    return read_tool.invoke({'file_path': file_name})
  except Exception as e:
    return f'Не удалось прочитать файл: {e}'

Работа с внешними источниками

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

@tool
def search_web(query: str) -> str:
  '''
  Поиск информации в интернете. Передает запрос, возвращает выдачу поисковика.\
  Полученный результат нужно обощить и передать пользователю.
  '''
  result = search.run(query)
  writer_tool = file_tools.get_tools()[-2]
  check_notes = file_tools.get_tools()[-1]
  n = len(check_notes.invoke({}).split('\n'))
  writer_tool.invoke({'file_path': f'Запрос: {query}' + f' {n}' + '.txt', 'text': result})
  # иногда запросы исчезают из нейронки. Чтобы это как то обойти, создается
  # заметка
  return result

In [ ]:
# Инструменты для поиска и саммаризации видеороликов
from langchain_community.tools import YouTubeSearchTool
from youtube_transcript_api import YouTubeTranscriptApi
from youtubesearchpython import Video, ResultMode
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document
from datetime import datetime, timedelta
import json


video_template = '''
Название ролика: {}

Канал: {}

Продолжительность ролика: {}

Дата публикации: {}

Описание видео: {}


'''

youtube = YouTubeSearchTool() # принимает запрос и возращает ссылки
sum_chain = load_summarize_chain(giga, chain_type='map_reduce')


@tool
def find_video(query: str) -> str:
  '''
  Поиск видео на YouTube. Возвращает название видео, продолжительность, название\
  канала и краткое содержание (если есть субтитры).
  '''
  links = youtube.run(query)
  links = json.loads(links.replace('\'', '"'))
  video_result = ''

  for link in links:
    video_id = link.split('=')[1]
    video_id = video_id.split('&')[0]

    try:
      youtube_data = YouTubeTranscriptApi.get_transcript(video_id, languages=['ru', 'en'])
      text = []

      for data in youtube_data:
        text.append(data['text']) # отбрасываются таймкоды от текста
        script = ' '.join(text)
      doc = Document(page_content=script)
      split_docs = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents([doc])
      res = sum_chain.run(split_docs)

    except Exception as e:
      res = 'Не удалось извлечь субтитры'

    video_info = Video.getInfo(video_id, mode=ResultMode.json)
    title = video_info['title']
    channel = video_info['channel']['name']

    # конвертация времени в чч:мм:сс
    duration_seconds = int(video_info['duration']['secondsText'])
    duration_str = str(timedelta(seconds=duration_seconds))

    # дата публикации в дд:мм:гг
    publish_date = datetime.strtime(video_info['publishDate'], '%Y-%m-%T%H:%M:%S%z')
    publish_date_str = publish_date.strftime('%d.%m.%y')

    video_result += video_template.format(title, channel, duration_str, publish_date_str, res)

  writer_tool = file_tools.get_tools()[-2]
  check_notes = file_tools.get_tools()[-1]

  n = len(check_notes.invoke({}).split('\n'))
  writer_tool.invoke({'file_path': f'Видео: {query}' + f' {n}' + '.txt', 'text': video_result})

  return video_result

In [ ]:
@tool
def memory_clearning() -> None:
  '''
  Очищает историю диалога, позволяя начать диалог пользователя с ассистентом с нуля.
  '''
  global chat_history_m
  chat_history = []

Создание агента

In [ ]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent


tools = [get_list_of_notes, write_note, read_note, find_video, search_web, memory_clearning]
agent = create_gigachat_functions_agent(giga, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
chat_history = []

ImportError: cannot import name 'create_gigachat_functions_agent' from 'langchain.agents' (/usr/local/lib/python3.10/dist-packages/langchain/agents/__init__.py)

Графический интерфейс в gradio

In [ ]:
import gradio as gr
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


def respond(message, chat_history):
  result = agent_executor.invoke({
    'chat_history': chat_history_m,
    'input': message
  })
  chat_history_m.append(HumanMessage(content=message))
  chat_history_m.append(AIMessage(content=result['output']))
  chat_history.append((message, result['output']))
  return '', chat_history


def main():
  with gr.Blocks() as demo:
    with gr.Column():
      chatbot = gr.Chatbot()
      msg = gr.Textbox()
      msg.submit(respond, [msg, chatbot], [msg, chatbot])
    demo.launch(debug=True)


if __name__ == '__main__':
  main()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f82625a998ed47a514.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-22-e03041d5fe94>:53: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = sum_chain.run(split_docs)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 622, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 2013, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1566, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f82625a998ed47a514.gradio.live


In [ ]:
search_web('Сколько параметров у гигачата')

'snippet: А теперь мы представляем новую нейросетевую модель, у которой более 29 миллиардов параметров. Она успешно прошла тест на ЕГЭ и показала отличные результаты в сравнении с другими системами., title: GigaChat Pro. Технические детали, сдача ЕГЭ и сравнение с конкурентами, link: https://habr.com/ru/companies/sberdevices/articles/780334/, snippet: Модель с 7 млрд параметров и увеличенным контекстом в 32 тыс. токенов. Идеально подойдёт для использования в задачах, предполагающих необходимость передать в запрос большой объем ..., title: GigaChat обогнал по качеству ChatGPT и ... - Habr, link: https://habr.com/ru/companies/sberdevices/articles/790470/, snippet: У модели есть инстаграм-аккаунт — berlin.1969. В профиле говорится, что мужчину зовут Эрнст Халимов. До 2021 года он хранил молчание. ... Сколько на самом деле стоит советская утварь из бабушкиного ..., title: Гигачад: что за мем с качком, откуда он взялся и при чем тут инцелы, link: https://journal.tinkoff.ru/is-gigachad-real/